In [1]:
%run deps.py
%matplotlib inline

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [71]:
protvec_model = biovec.models.load_protvec('./trained_models/swissprot_reviewed_protvec')

def seq2protvec(seq):
    return sum(protvec_model.to_vecs(seq))

def get_labels(labels_col):
    le = LabelEncoder()
    return le.fit_transform(labels_col)

def get_protvec(data):
    seq2pv = data.apply(seq2protvec)
    seq2pv = list(seq2pv)
    return seq2pv

def meas2binary(meases):
    return pd.Series(map(lambda x: 1 if x >= log_meas(500) else 0, meases))

def xgb_regression(data, target):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25, random_state=442)
    
    xgbr = xgb.XGBRegressor(max_depth=9, learning_rate=0.1, n_estimators=750, gamma=0, min_child_weight=5,
                            reg_lambda = 0.3, subsample=0.9, colsample_bytree = 0.9, nthread=15, silent=False)
    
    xgbr.fit(X_train, y_train)
    
    preds = xgbr.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    f1 = f1_score(meas2binary(y_test), meas2binary(preds))
    print("RMSE on test: {0:.4f}".format(mse))
    print("F1-score on test:{}".format(f1))
    return mse, f1

def get_rmse_of_models(performance, my_performance_mse, my_performance_f1, alleles):
    
    """
    arguments:
    
    performance - 
    alleles - list of alleles names
    my_performnace - 
    
    --------
    function get_rmse_of_models extract dataset of specific allele from performance dataset, calculates rmse on 
    each allele and stores it in dicts
    --------
    returns: 3 tuples
    
    netmhc_rmse - tuple of rmse of netmhc on every alelle
    netmhcpan_rmse - tuple of rmse of netmhcpan on every alelle
    pmbec_rmse - tuple of rmse of smm_pmbec on every alelle
    
    """
    
    netmhc_rmse = {}
    netmhcpan_rmse = {}
    pmbec_rmse = {}
    mhc_rmse = {}
    
    netmhc_f1 = {}
    netmhcpan_f1 = {}
    pmbec_f1 = {}
    mhc_f1 = {}
    
    for index, cur_al in enumerate(alleles):
        
        cur_alelle = select_by_allele(performance, cur_al)
        print("\n#{:d} {}:".format(index, cur_al))
        print("Shapes: ", cur_alelle.shape, cur_alelle.meas.shape)
        if(len(cur_alelle) == 0):
            continue
        netmhc_rmse[cur_al] = mean_squared_error(cur_alelle.meas, cur_alelle.netmhc)
        netmhcpan_rmse[cur_al] = mean_squared_error(cur_alelle.meas, cur_alelle.netmhcpan)
        pmbec_rmse[cur_al] = mean_squared_error(cur_alelle.meas, cur_alelle.smmpmbec_cpp)
        
        netmhc_f1[cur_al] = f1_score(meas2binary(cur_alelle.meas), meas2binary(cur_alelle.netmhc))
        netmhcpan_f1[cur_al] = f1_score(meas2binary(cur_alelle.meas), meas2binary(cur_alelle.netmhcpan))
        pmbec_f1[cur_al] = f1_score(meas2binary(cur_alelle.meas), meas2binary(cur_alelle.smmpmbec_cpp))
        
    
    netmhc_df = pd.DataFrame.from_dict(netmhc_rmse, orient='index')
    netmhcpan_df = pd.DataFrame.from_dict(netmhcpan_rmse, orient='index')
    pmbec_df = pd.DataFrame.from_dict(pmbec_rmse, orient='index')
    xgb_df = pd.DataFrame.from_dict(my_performance_mse, orient='index')

    models_perf_mse = pd.concat([xgb_df, netmhc_df, netmhcpan_df, pmbec_df], axis=1)
    models_perf_mse.columns=["mhystic", "netmhc", "netmhcpan", "pmbec"]
    
    netmhc_df = pd.DataFrame.from_dict(netmhc_f1, orient='index')
    netmhcpan_df = pd.DataFrame.from_dict(netmhcpan_rmse, orient='index')
    pmbec_df = pd.DataFrame.from_dict(pmbec_f1, orient='index')
    xgb_df = pd.DataFrame.from_dict(my_performance_f1, orient='index')

    models_perf_f1 = pd.concat([xgb_df, netmhc_df, netmhcpan_df, pmbec_df], axis=1)
    models_perf_f1.columns=["mhystic", "netmhc", "netmhcpan", "pmbec"]
    
    return models_perf_mse, models_perf_f1


def evaluate_pv_by_allele(dataset, alleles):
    
    """
    arguments:
    
    alleles - list of alleles names
    --------
    function evaluate_by_allele extract dataset of specific allele from Bdata in which there are sequence and affinity 
    using select_by_allele function, saves it in csv, then constructing word2vec representation of this 
    sequences and evaluates xgb model, which return rmse on this allele.
    --------
    returns: 
    
    mhc_rmse tuple of rmse on every allele in alleles
    """
    
    mhc_rmse = {}
    mhc_f1 = {}
    
    for index,cur_al in enumerate(alleles):
        
        current_allele = select_by_allele(dataset, cur_al)
        
        if(len(current_allele) == 1):
            continue
            
        cur_al_affinity = current_allele.meas
        
        #pv_protein = get_protvec(current_allele.protein)
        pv_peptide = get_protvec(current_allele.sequence)
        
        #pv_mean = np.array([i/2 for i in np.add(pv_protein, pv_peptide)])
        #pv_res = np.subtract(pv_protein, pv_peptide)
        #pv_sum = np.add(pv_protein, pv_peptide)
        
        
        print("\n#{:d} {}:".format(index, cur_al))
        print("Shapes: ", current_allele.shape, cur_al_affinity.shape)
        
        allele_rmse, allele_f1 = xgb_regression(pv_peptide, cur_al_affinity)
        mhc_rmse[cur_al] = allele_rmse
        mhc_f1[cur_al] = allele_f1
    
    return mhc_rmse, mhc_f1

def plot_results(results, description):
    
    """
    results - DataFrame with columns "mhystic", "netmhc", "netmhcpan", "pmbec"
    
    """
    
    allele_rmse_figure = plt.figure(figsize=(48, 27))
    plt.title("RMSE.{}".format(description))
    plt.xlabel("$RMSE$")
    plt.ylabel("Alleles")

    ind = np.arange(0, 4*len(list(results.index)), 4) 
    width = 0.6
    ys = plt.yticks(ind, list(results.index), rotation=0)
    plt.locator_params(nbins=len(results.mhystic))

    xgb_rects = plt.barh(ind, list(results.mhystic), width, align='center', color='red', alpha=0.3, label = 'MHystic', edgecolor='w')
    plt.barh(ind + width , list(results.netmhc), width, align='center', color='yellow', alpha=0.3, label='netmhc')
    plt.barh(ind + 2*width, list(results.netmhcpan), width, align='center', color='blue', alpha=0.3, label='netmhcpan')
    plt.barh(ind + 3*width, list(results.pmbec), width, align='center', color='black', alpha=0.3, label='pmbec')

    plt.legend(fontsize='xx-large')


    rounded_rmse_xgb = ["{0:.5f}".format(i) for i in list(results.mhystic)]

    for cur_rect, value in zip(xgb_rects.patches, rounded_rmse_xgb):
        plt.text(cur_rect.get_width() +0.001, cur_rect.get_y() - 0.5, value, ha='center', va='bottom', fontsize=15)

    
    allele_rmse_figure.savefig('rmse {}.png'.format(description), dpi=allele_rmse_figure.dpi)

INFO : loading Word2Vec object from ./trained_models/swissprot_reviewed_protvec
INFO : setting ignored attribute syn0norm to None
INFO : setting ignored attribute cum_table to None
INFO : loaded ./trained_models/swissprot_reviewed_protvec


In [3]:
Bdata = pd.read_csv("./data/bdata.csv")
Bdata.meas = [log_meas(i) for i in Bdata.meas]
data9mers = Bdata[Bdata.peptide_length == 9].reset_index()

hla_abce = select_hla(data9mers)
seq_dict = pd.read_csv("./data/mhc_seq_imghtla.csv")
common = set(seq_dict.mhc).intersection(set(hla_abce.mhc))
seq_dict = seq_dict[seq_dict["mhc"].isin(common)].reset_index(drop=True)
hla_abce = hla_abce.iloc[np.array(np.where(hla_abce.mhc.apply(lambda x:x in common))).flatten()].reset_index(drop=True)
hla_abce["protein"] = [seq_dict.loc[seq_dict.mhc==i].sequence.values[0] for i in hla_abce.mhc]

In [17]:
proteins2pv = get_protvec(hla_abce.protein)
peptides2pv = get_protvec(hla_abce.sequence)

In [29]:
pv_res = np.subtract(proteins2pv, peptides2pv)
pv_div = np.divide(proteins2pv, peptides2pv)
pv_sum = np.add(proteins2pv, peptides2pv)
pv_mean = np.array([i/2 for i in pv_sum])

In [ ]:
performance_data = pd.read_csv("./data/model_performance_logged.csv").drop("Unnamed: 0", axis=1)
performance_data.tail()

unify_alleles = lambda x: re.sub('[-|*]', '', x)

performance_data.mhc = performance_data.mhc.apply(unify_alleles)

common_alleles = list(set(hla_abce.mhc).intersection(performance_data.mhc))

errors = evaluate_pv_by_allele(hla_abce, common_alleles)
errors_rmse = OrderedDict(sorted(errors[0].items(), key=lambda t: t[0]))
errors_f1 = OrderedDict(sorted(errors[1].items(), key=lambda t: t[0]))
performance_df_mse, performance_df_f1 = get_rmse_of_models(performance_data, errors[0], errors[1], common_alleles)


#0 HLAA2301:
Shapes:  (1784, 8) (1784,)
RMSE on test: 0.0670
F1-score on test:0.15151515151515152

#1 HLAB2705:
Shapes:  (2587, 8) (2587,)
RMSE on test: 0.0412
F1-score on test:0.18867924528301885

#2 HLAA0101:
Shapes:  (3680, 8) (3680,)
RMSE on test: 0.0503
F1-score on test:0.056338028169014086

#3 HLAB0801:
Shapes:  (2817, 8) (2817,)
RMSE on test: 0.0641
F1-score on test:0.33628318584070793

#4 HLAB5101:
Shapes:  (2031, 8) (2031,)
RMSE on test: 0.0351
F1-score on test:0.07547169811320754

#5 HLAB5801:
Shapes:  (2757, 8) (2757,)
RMSE on test: 0.0583
F1-score on test:0.2372881355932203

#6 HLAA0203:
Shapes:  (4427, 8) (4427,)
RMSE on test: 0.0944
F1-score on test:0.49002849002849

#7 HLAB4002:
Shapes:  (572, 8) (572,)
RMSE on test: 0.0979
F1-score on test:0.06779661016949153

#8 HLAB0803:
Shapes:  (451, 8) (451,)


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


RMSE on test: 0.0144
F1-score on test:0.0

#9 HLAA3001:
Shapes:  (2426, 8) (2426,)
RMSE on test: 0.0920
F1-score on test:0.2453531598513011

#10 HLAB1517:
Shapes:  (1430, 8) (1430,)
RMSE on test: 0.1074
F1-score on test:0.19858156028368795

#11 HLAA0206:
Shapes:  (3732, 8) (3732,)
RMSE on test: 0.0978
F1-score on test:0.5378590078328982

#12 HLAA0201:
Shapes:  (8826, 8) (8826,)
RMSE on test: 0.0878
F1-score on test:0.47283406754772395

#13 HLAA3002:
Shapes:  (1202, 8) (1202,)
RMSE on test: 0.0752
F1-score on test:0.20512820512820512

#14 HLAA0202:
Shapes:  (2465, 8) (2465,)
RMSE on test: 0.0937
F1-score on test:0.5979020979020979

#15 HLAB0802:
Shapes:  (998, 8) (998,)


/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


RMSE on test: 0.0090
F1-score on test:0.0

#16 HLAB3801:
Shapes:  (491, 8) (491,)
RMSE on test: 0.0455
F1-score on test:0.7088607594936709

#17 HLAB4601:
Shapes:  (1784, 8) (1784,)
RMSE on test: 0.0147
F1-score on test:0.1739130434782609

#18 HLAA3301:
Shapes:  (1929, 8) (1929,)
RMSE on test: 0.0622
F1-score on test:0.19469026548672566

#19 HLAB5301:
Shapes:  (937, 8) (937,)
RMSE on test: 0.0737
F1-score on test:0.5

#20 HLAA6801:
Shapes:  (2036, 8) (2036,)
RMSE on test: 0.0880
F1-score on test:0.4054794520547945

#21 HLAB1501:
Shapes:  (3797, 8) (3797,)
RMSE on test: 0.0686
F1-score on test:0.2040816326530612

#22 HLAA6802:
Shapes:  (3672, 8) (3672,)
RMSE on test: 0.0649
F1-score on test:0.24305555555555555

#23 HLAB1801:
Shapes:  (2190, 8) (2190,)
RMSE on test: 0.0390
F1-score on test:0.03333333333333333

#24 HLAB0702:
Shapes:  (3619, 8) (3619,)
RMSE on test: 0.0610
F1-score on test:0.4509283819628648

#25 HLAA2603:
Shapes:  (517, 8) (517,)
RMSE on test: 0.0412
F1-score on test:0.0



/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


RMSE on test: 0.0281
F1-score on test:0.0

#39 HLAA6901:
Shapes:  (2557, 8) (2557,)


In [ ]:
plot_results(performance_df_mse.sort('mhystic', ascending=True), "RMSE, pv, peptide")

In [ ]:
plot_results(performance_df_f1.sort('mhystic', ascending=True), "F1, pv, peptide")